# Prototipo de Dashboard (Fase 2)

Este notebook simula las pruebas realizadas para la creación de `main_dashboard.py`. 

**Objetivo:** Probar la carga de datos, la lógica de filtrado y las visualizaciones de Plotly antes de pasarlas a Streamlit.

## 1. Importaciones y Configuración

In [1]:
import pandas as pd
import plotly.express as px
import os

# NOTA: No importamos 'streamlit' aquí, para probar la lógica.

## 2. Carga de Datos

Probamos la función de carga de datos (`load_data`) del script. Usando la ruta al archivo procesado del pipe line.

In [9]:
BASE_DIR = os.path.abspath('..') 
DATA_PROCESS_DIR = os.path.join(BASE_DIR, 'data', 'process')
PROCESSED_FILE = os.path.join(DATA_PROCESS_DIR, 'procesados_movies.csv')

def load_data(file_path):
    """
    Prueba de la función de carga.
    """
    try:
        df = pd.read_csv(file_path, encoding='latin1') 
        return df
    except FileNotFoundError:
        print(f"Error: No se encontró el archivo 'procesados_movies.csv' en {file_path}")
        return None

# Cargamos los datos
df_procesado = load_data(PROCESSED_FILE)

if df_procesado is not None:
    print(f"Datos cargados exitosamente: {df_procesado.shape}")
    display(df_procesado.head())

Datos cargados exitosamente: (27278, 47)


,movieid,title,genres,no_genres_listed,Action,Adventure,Animation,Children,Comedy,Crime,...,2007,2008,2009,2010,2011,2012,2013,2014,2015,tag
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,0,0,1,1,1,1,0,...,3.672973,3.740781,3.766777,3.856772,3.861689,3.931421,3.946274,3.954945,3.860412,"Watched, computeranimation, Disneyanimatedfeat..."
1,2,Jumanji (1995),Adventure|Children|Fantasy,0,0,1,0,1,0,0,...,2.923704,3.059659,3.044865,3.159884,3.178733,3.201903,3.266667,3.346552,3.128049,"timetravel, adaptedfrombook, boardgame, childh..."
2,3,Grumpier Old Men (1995),Comedy|Romance,0,0,0,0,0,1,0,...,2.990909,3.112245,3.102837,2.986364,3.051724,2.799451,3.064103,3.048387,2.854167,"oldpeoplethatisactuallyfunny, sequelfever, gru..."
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,0,0,0,0,0,1,0,...,2.310811,2.523810,2.500000,2.833333,2.944444,2.545455,3.187500,2.357143,2.000000,"chickflick, revenge, characters, CLV"
4,5,Father of the Bride Part II (1995),Comedy,0,0,0,0,0,1,0,...,2.633333,2.725849,2.864662,2.812500,2.913462,2.609804,2.770936,2.897059,2.842105,"DianeKeaton, family, sequel, SteveMartin, wedd..."


## 3. Función Auxiliar 

Probamos la función `get_dynamic_columns` para extraer correctamente los géneros (One-Hot) y los años (Pivot).

In [10]:
def get_dynamic_columns(df):
    """
    La función 'get_dynamic_columns' inspecciona el df y
    extrae las columnas de género y las de año desde (Pivot).
    """
    base_cols = ['movieid', 'title', 'genres', 'rating_promedio', 'rating_conteo', 'tags_agrupados']
    year_cols = [col for col in df.columns if col.isdigit() and len(col) == 4]
    genre_cols = [col for col in df.columns if col not in base_cols and col not in year_cols]
    
    return genre_cols, year_cols


genre_columns, year_columns = get_dynamic_columns(df_procesado)

print(f"Se encontraron {len(genre_columns)} columnas de género.")
print(f"Ejemplo: {genre_columns[:5]}")

print(f"\nSe encontraron {len(year_columns)} columnas de año.")
print(f"Ejemplo: {year_columns[:5]}")

Se encontraron 21 columnas de género.
Ejemplo: ['no_genres_listed', 'Action', 'Adventure', 'Animation', 'Children']

Se encontraron 21 columnas de año.
Ejemplo: ['1995', '1996', '1997', '1998', '1999']


## 4. Simulación  Filtros

En Streamlit, estos valores vendrían de `st.slider` o `st.multiselect`.

In [11]:
selected_genres = ['Action', 'SciFi']
rating_slider = (3.5, 5.0) # (min, max)
total_ratings_slider = 1000 # Mínimo 1000 calificaciones

print(f"Simulando filtros:")
print(f"- Géneros: {selected_genres}")
print(f"- Rango de Rating: {rating_slider}")
print(f"- Mínimo de Calificaciones: {total_ratings_slider}")

Simulando filtros:
- Géneros: ['Action', 'SciFi']
- Rango de Rating: (3.5, 5.0)
- Mínimo de Calificaciones: 1000


## 5. Prueba de la Lógica de Filtrado

Aplicamos los filtros simulados a nuestro DataFrame para ver el resultado.

In [12]:
df_filtrado = df_procesado.copy()
df_filtrado = df_filtrado[
    (df_filtrado['rating_promedio'] >= rating_slider[0]) &
    (df_filtrado['rating_promedio'] <= rating_slider[1])
]

df_filtrado = df_filtrado[
    df_filtrado['rating_conteo'] >= total_ratings_slider
]

if selected_genres:
    for genre in selected_genres:
        df_filtrado = df_filtrado[df_filtrado[genre] == 1]

print(f"Filas antes: {len(df_procesado)} | Filas después: {len(df_filtrado)}")
display(df_filtrado.head())

Filas antes: 27278 | Filas después: 71


,movieid,title,genres,no_genres_listed,Action,Adventure,Animation,Children,Comedy,Crime,...,2007,2008,2009,2010,2011,2012,2013,2014,2015,tag
257,260,Star Wars Episode IV A New Hope (1977),Action|Adventure|Sci-Fi,0,1,1,0,0,0,0,...,4.035646,4.050584,4.041898,4.084775,4.097885,4.096226,4.149308,4.052098,3.897613,"1970s, fantasy, scifi, epic, exciting, galacti..."
476,480,Jurassic Park (1993),Action|Adventure|Sci-Fi|Thriller,0,1,1,0,0,0,0,...,3.474814,3.562008,3.578277,3.585590,3.637795,3.735090,3.800918,3.775970,3.851027,"adaptedfrombook, authorMichaelCrichton, herdso..."
537,541,Blade Runner (1982),Action|Sci-Fi|Thriller,0,1,0,0,0,0,0,...,4.048155,4.084385,3.982759,4.030967,4.023373,4.073712,4.089443,4.096392,4.137283,"cyberpunk, dystopia, memory, PhilipKDick, scif..."
583,589,Terminator 2 Judgment Day (1991),Action|Sci-Fi,0,1,0,0,0,0,0,...,3.762672,3.892332,3.921798,3.941641,3.897656,3.996193,4.000000,3.966258,3.922551,"dystopia, scifi, timetravel, androidscyborgs, ..."
1104,1127,"Abyss, The (1989)",Action|Adventure|Sci-Fi|Thriller,0,1,1,0,0,0,0,...,3.636364,3.740769,3.608059,3.648536,3.658163,3.728916,3.750000,3.782000,3.789474,"Underwater, scifi, aliens, submarine, firstcon..."


## 6. Prueba de Visualizaciones (Plotly)

Probamos cada gráfico individualmente usando el `df_filtrado`.

### 6.1 Gráfico de Barras: Top 10 Popularidad

In [13]:
df_top10 = df_filtrado.nlargest(10, 'rating_conteo')
fig_bar = px.bar(
    df_top10, x='rating_conteo', y='title', orientation='h',
    title='Top 10 Películas más Populares', hover_data=['rating_promedio', 'genres']
)
fig_bar.update_layout(yaxis={'categoryorder':'total ascending'})


fig_bar.show()

### 6.2 Gráfico de Dispersión: Rating vs. Popularidad

In [14]:
fig_scatter = px.scatter(
    df_filtrado, x='rating_conteo', y='rating_promedio',
    title='Rating vs. Popularidad', hover_data=['title', 'genres'],
    color='rating_promedio', size='rating_conteo' 
)
fig_scatter.show()

### 6.3 Gráfico de Líneas: Evolución del Rating

Simulamos que el usuario selecciona una película (`selected_movie_title`) del `df_filtrado`.

In [15]:
# Simulamos la selección de la primera película de la lista filtrada
if not df_filtrado.empty:
    selected_movie_title = df_filtrado.iloc[0]['title']
    print(f"Simulando selección de: {selected_movie_title}")

    movie_data = df_filtrado[df_filtrado['title'] == selected_movie_title].iloc[0]
    evolution_data = movie_data[year_columns]
    evolution_data = evolution_data[evolution_data > 0]
    
    if not evolution_data.empty:
        df_evo = pd.DataFrame({
            'Anio': evolution_data.index.astype(int),
            'Rating Promedio': evolution_data.values
        })
        
        fig_line = px.line(
            df_evo, x='Anio', y='Rating Promedio',
            title=f"Evolución del Rating para: {selected_movie_title}", markers=True
        )
        fig_line.update_xaxes(type='linear')
        fig_line.show()
    else:
        print(f"No hay datos de evolución para {selected_movie_title}")
else:
    print("No hay películas en el DataFrame filtrado para probar el gráfico de evolución.")

Simulando selección de: Star Wars Episode IV  A New Hope (1977)
